In [133]:
import pandas as pd


In [134]:
path_contract = r'C:\Users\yuriy\Desktop\equipment\org_files\Таблица по законтрактованному объему 02.11.2023.xlsx'
path_eqp_unit = r'C:\Users\yuriy\Desktop\equipment\org_files\Попозиционный учет законтрактованного 2023.11.02.xlsx'
path_pipe = r'C:\Users\yuriy\Desktop\equipment\Трубы\трубы.xlsx'
path_pipe_tsm = r'C:\Users\yuriy\Desktop\equipment\org_files\1 - TSM ENERJI - общая сводная учета данных.xlsx'

df_contract = pd.read_excel(path_contract, sheet_name='Contract')
df_eqp = pd.read_excel(path_eqp_unit, sheet_name='Общий свод', skiprows=1)
df_pipe = pd.read_excel(path_pipe, sheet_name='Трубопроводы', skiprows=2)
df_zra = pd.read_excel(path_pipe, sheet_name='ЗРА', skiprows=2, usecols=['Номер контракта'])
df_pipe_tsm = pd.read_excel(path_pipe_tsm, sheet_name='Учет основной', skiprows=3)


C:\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Cell BD123545 is marked as a date but the serial value 3086000 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():


In [141]:
path_pipe = r'C:\Users\yuriy\Desktop\equipment\non project works\трубы_clean.xlsx'
df_pipe_clean = pd.read_excel(path_pipe)

In [ ]:
print('Cols трубы:\n', df_pipe.columns, '\n')
# print('Cols законтрактованному объему:\n', df_contract.columns, '\n')
# print('Cols учет трубы TSM:\n', df_pipe_tsm.columns)

In [150]:
df_pipe_clean = df_pipe_clean.drop(['Номер контракта'], axis=1)
merged_df = df_pipe_clean.merge(df_contract[['Номер контракта', 'ЭБ №1', 'ЭБ №2', 'ЭБ №3', 'ЭБ №4']], left_on='Номерконтракта', right_on='Номер контракта', how='left')
merged_df = merged_df.drop(['Номер контракта', 'ЭБ 1', 'ЭБ 2', 'ЭБ 3', 'ЭБ 4'], axis=1).rename(columns={
    'ЭБ №1':'ЭБ 1',
    'ЭБ №2':'ЭБ 2',
    'ЭБ №3':'ЭБ 3',
    'ЭБ №4':'ЭБ 4'
})
# merged_df2 = merged_df.merge(df_pipe_tsm[['','']], left_on='', right_on='', how='left')

In [ ]:
#zra
df_contract_zra = df_contract.loc[df_contract['Наименование поставки'].str.contains('задвиж|клапан',  na=False, case=False)]
df_contract_zra = df_contract_zra[~df_contract_zra['Статус подписания 27.10.2023'].str.contains('АННУЛИРОВАН|расторжение')]
df_contract_zra

In [ ]:
# only_pipe = df_pipe[~df_pipe['Номерконтракта'].isin(df_contract_zra['Номер контракта'])]
# only_zra = df_contract_zra[~df_contract_zra['Номер контракта'].isin(df_pipe['Номерконтракта'])]
# 
# print('only pipe:\n', only_pipe['Номерконтракта'], '\n')
# print('only zra:\n', only_zra['Номер контракта'])


In [ ]:
df_pipe2 = df_contract.loc[df_contract['Наименование поставки'].str.contains('труба|трубопроводы|труба',  na=False, case=False)]
df_pipe2 = df_pipe2[~df_pipe2['Статус подписания 27.10.2023'].str.contains('АННУЛИРОВАН|расторжение')]
only_pipe2 = df_pipe2[~df_pipe2['Номер контракта'].isin(df_pipe['Номерконтракта'])]
only_pipe = df_pipe[~df_pipe['Номерконтракта'].isin(df_pipe2['Номер контракта'])]

only_pipe2

In [ ]:
path_upr = r'C:\Users\yuriy\Desktop\equipment\org_files\Управленческий учет (еженедельный отчет).xlsx'
df_mng_accounting = pd.read_excel(path_upr, sheet_name='Оборудование', skiprows=1)
df_mng_accounting

In [ ]:
only_zra = df_zra[~df_zra['Номер контракта'].isin(df_mng_accounting['Номерконтракта'])]
only_mng_acc = df_mng_accounting[~df_mng_accounting['Номерконтракта'].isin(df_zra['Номер контракта'])]

mng_acc = df_mng_accounting[df_mng_accounting['Номерконтракта'].isin(df_zra['Номер контракта'])]

df_contract2 = df_contract[['п/п', 'Номер лота', 'Номер контракта', 'Поставщик из ЭДО', 'Наименование поставки', 'Сумма контракта, без НДС', 
                            'ЭБ №1', 'ЭБ №2', 'ЭБ №3', 'ЭБ №4', 'Статья']]


In [ ]:
merged_test = df_zra.merge(df_contract2, on='Номер контракта', how='inner')

merged_test

In [ ]:
# contract_replace = {
#     'FT-04-22-1685': '2212-12-AKU-22-046/046м/FT-04-22-1685',
#     'FT-04-23-77': 'FT-04-23-77 от 30.01.2023',
#     'FT-04-23-704': 'FT-04-23-704 от 23.05.2023',
#     'FT-04-23-646': 'FT-04-23-646 от 05.05.2023',
#     'FT-04-23-781': 'FT-04-23-781 от 23.07.2023',
# }
# 
# for val in df_pipe['Номерконтракта']:
#     if val in contract_replace.keys():
#         contract_val = contract_replace[val]
#     else:
#         contract_val = val
#     df = df_pipe_tsm[df_pipe_tsm['Контракт'] == contract_val]
#     merged_df.loc[ merged_df['Номерконтракта'] == val ,'proekt_tonn'] = df['Масса общая, тн.'].sum()
#     merged_df.loc[ merged_df['Номерконтракта'] == val ,'postavleno_tonn'] = df['Поставлено, тн.'].sum()
#     merged_df.loc[ merged_df['Номерконтракта'] == val ,'montaj_tonn'] = df['Монтаж BR'].sum()
#     
#     unique_values = df['Здание'].unique()
#     result_string = ', '.join(unique_values)
#     merged_df.loc[ merged_df['Номерконтракта'] == val ,'zdanie'] = result_string
#     
#     # print(df['Масса общая, тн.'])
#     del df
#     
# merged_df


# df = df_pipe_tsm[df_pipe_tsm['Контракт'] == 'FT-04-21-889']
# sum_df = df['Масса общая, тн.'].sum()
# sum_df

#exist 
# FT-04-21-889
# FT-04-22-1692
# FT-04-21-642
# FT-04-23-664

# does not exist
# FT-04-21-626
# FT-04-23-665
# FT-04-21-890
# FT-04-22-1685 -> 2212-12-AKU-22-046/046м/FT-04-22-1685
# FT-04-20-76
# FT-04-23-912
# FT-04-23-77 -> FT-04-23-77 от 30.01.2023
# FT-04-23-828
# FT-04-21-1058
# FT-04-23-144
# FT-04-21-702
# FT-04-23-933
# FT-04-23-68
# FT-04-23-704 -> FT-04-23-704 от 23.05.2023
# FT-04-22-922
# FT-04-23-143
# FT-04-21-1060
# FT-04-23-646  -> FT-04-23-646 от 05.05.2023
# FT-04-23-781 -> FT-04-23-781 от 23.07.2023

In [ ]:
df_pipe_tsm[df_pipe_tsm['Контракт'] == 'FT-04-21-642']

In [152]:
df_bloks = df_contract.loc[:,['Номер контракта', 'блоки ']]

In [153]:
# for val in merged_df['Номерконтракта']:
#     df = df_contract[df_contract['Номер контракта'] == val]
#     merged_df.loc[ merged_df['Номерконтракта'] == val ,'bloki'] = df['блоки ']

merged_df_final = merged_df.merge(df_bloks, left_on='Номерконтракта', right_on='Номер контракта', how='inner').drop(['Номер контракта'], axis=1)

merged_df_final

,№,Лот,Номерконтракта,Дата контракта,Поставщик,Предмет поставки,Стоимость контракта,Предусмотренный аванс,Тонн,Смонтировано,...,Оплачено за поставку оборудования,Статья,Объекты,bloki,блоки _x,ЭБ 1,ЭБ 2,ЭБ 3,ЭБ 4,блоки _y
0,412,AKU-21-021/021m,FT-04-21-889,2021-10-25,КОНЦЕРН ТИТАН-2 АО,Поставка трубопроводов турбинного острова для ...,1.209500e+08,3.628500e+07,NaN,NaN,...,NaN,Оборудование и материалы,NaN,NaN,"1,2,3,4",3.023750e+07,3.023750e+07,30237500.0,30237500.0,"1,2,3,4"
1,413,AKU-20-207/207м,FT-04-21-626,2021-06-15,КОНЦЕРН ТИТАН-2 АО,Трубопроводы ядерного острова,5.591670e+07,1.677501e+07,NaN,NaN,...,7182500.50,Оборудование и материалы,NaN,NaN,"1, 2",2.797751e+07,2.793918e+07,NaN,NaN,"1, 2"
2,414,AKU-21-113/113м,FT-04-22-1692,2022-12-12,КОНЦЕРН ТИТАН-2 АО,трубопроводы турбинного острова,3.985523e+07,9.060729e+06,NaN,NaN,...,NaN,Оборудование и материалы,NaN,NaN,1.2,1.992791e+07,1.992733e+07,NaN,NaN,1.2
3,415,AKU-22-282/282м,FT-04-23-665,2023-05-04,КОНЦЕРН ТИТАН-2 АО,трубопроводы турбинного острова,3.824765e+07,9.037067e+06,NaN,NaN,...,NaN,Оборудование и материалы,NaN,NaN,1.2,1.913798e+07,1.910967e+07,NaN,NaN,1.2
4,416,AKU-20-273/273м,FT-04-21-642,2021-06-30,КОНЦЕРН ТИТАН-2 АО,"Трубопроводов LAB, LBA",3.432932e+07,1.029880e+07,NaN,NaN,...,9794894.27,Оборудование и материалы,NaN,NaN,"1, 2",1.714690e+07,1.718242e+07,NaN,NaN,"1, 2"
5,417,AKU-22-281/281м,FT-04-23-664,2023-05-02,КОНЦЕРН ТИТАН-2 АО,трубопроводы турбинного острова,3.027153e+07,8.071123e+06,NaN,NaN,...,NaN,Оборудование и материалы,NaN,NaN,1.2,1.524803e+07,1.502350e+07,NaN,NaN,1.2
6,418,AKU-19-058/2/2м,FT-04-21-890,2021-10-18,КОНЦЕРН ТИТАН-2 АО,Поставка трубопроводов ядерного острова для со...,1.352748e+07,4.058243e+06,NaN,NaN,...,NaN,Оборудование и материалы,NaN,NaN,2,NaN,1.352748e+07,NaN,NaN,2
7,419,AKU-22-046/046м,FT-04-22-1685,2022-12-13,КОНЦЕРН ТИТАН-2 АО,"трубопроводы зданий 01UYX, 00UYB, 00UFC",3.213636e+06,9.640909e+05,NaN,NaN,...,625751.67,Оборудование и материалы,NaN,NaN,0,3.274509e+06,NaN,NaN,NaN,0
8,420,AKU-19-058,FT-04-20-76,2020-03-20,ТИТАН ТЕХНОЛОДЖИ ПАЙПЛАЙН ООО,Трубопроводы ЯО,1.247274e+07,3.741823e+06,NaN,NaN,...,3959016.20,Оборудование и материалы,NaN,NaN,1,1.396713e+07,0.000000e+00,0.0,0.0,1
9,454,AKU-22-283м,FT-04-23-912,2023-09-13,КОНЦЕРН ТИТАН-2 АО,трубопроводы турбинного острова,4.035195e+07,9.452929e+06,NaN,NaN,...,NaN,Материалы,NaN,NaN,1.2,2.017603e+07,2.017593e+07,NaN,NaN,1.2


In [ ]:
merged_df_final.to_excel(r'C:\Users\yuriy\Desktop\equipment\non project works\трубы_clean.xlsx', index=False, float_format="%.4f")


In [154]:
df2 = pd.DataFrame([])
for val in merged_df_final['Номерконтракта']:
    df = df_eqp[df_eqp['No Contract'] == val]
    filtered_df = df[['Lot', 'No Contract', 'Unit No.', ' Net weight  total, kg', 'Total without VAT, USD', 'Total with VAT, USD']]
    
    filtered_df.loc[:, ' Net weight  total, kg'] = filtered_df[' Net weight  total, kg'].astype(str)
    filtered_df.loc[:, 'Total without VAT, USD'] = filtered_df['Total without VAT, USD'].astype(str)
    filtered_df.loc[:, 'Total with VAT, USD'] = filtered_df['Total with VAT, USD'].astype(str)
    
    filtered_df.loc[:, ' Net weight  total, kg'] = pd.to_numeric(filtered_df[' Net weight  total, kg'].str.replace(',', '.', regex=False), errors='coerce')
    filtered_df.loc[:, 'Total without VAT, USD'] = pd.to_numeric(filtered_df['Total without VAT, USD'].str.replace(',', '.', regex=False), errors='coerce')
    filtered_df.loc[:, 'Total with VAT, USD'] = pd.to_numeric(filtered_df['Total with VAT, USD'].str.replace(',', '.', regex=False), errors='coerce')
    
    #weight
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'weight_u0'] = filtered_df.loc[filtered_df['Unit No.'] == 0,' Net weight  total, kg'].sum() / 1000
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'weight_u1'] = filtered_df.loc[filtered_df['Unit No.'] == 1,' Net weight  total, kg'].sum() / 1000
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'weight_u2'] = filtered_df.loc[filtered_df['Unit No.'] == 2,' Net weight  total, kg'].sum() / 1000
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'weight_u3'] = filtered_df.loc[filtered_df['Unit No.'] == 3,' Net weight  total, kg'].sum() / 1000
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'weight_u4'] = filtered_df.loc[filtered_df['Unit No.'] == 4,' Net weight  total, kg'].sum() / 1000
    #total_weight
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'total_weight'] = (merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'weight_u0'] +  
                                                                                     merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'weight_u1'] + 
                                                                                     merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'weight_u2'] + 
                                                                                     merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'weight_u3'] + 
                                                                                     merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'weight_u4'])
    
    #cost no VAT
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_u0'] = filtered_df.loc[filtered_df['Unit No.'] == 0,'Total without VAT, USD'].sum() 
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_u1'] = filtered_df.loc[filtered_df['Unit No.'] == 1,'Total without VAT, USD'].sum() 
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_u2'] = filtered_df.loc[filtered_df['Unit No.'] == 2,'Total without VAT, USD'].sum() 
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_u3'] = filtered_df.loc[filtered_df['Unit No.'] == 3,'Total without VAT, USD'].sum() 
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_u4'] = filtered_df.loc[filtered_df['Unit No.'] == 4,'Total without VAT, USD'].sum() 
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'total_cost'] = (merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_u0'] +  
                                                                                     merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_u1'] + 
                                                                                     merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_u2'] + 
                                                                                     merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_u3'] + 
                                                                                     merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_u4'])
    #cost with VAT
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_vat_u0'] = filtered_df.loc[filtered_df['Unit No.'] == 0,'Total with VAT, USD'].sum() 
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_vat_u1'] = filtered_df.loc[filtered_df['Unit No.'] == 1,'Total with VAT, USD'].sum()
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_vat_u2'] = filtered_df.loc[filtered_df['Unit No.'] == 2,'Total with VAT, USD'].sum()
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_vat_u3'] = filtered_df.loc[filtered_df['Unit No.'] == 3,'Total with VAT, USD'].sum()
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_vat_u4'] = filtered_df.loc[filtered_df['Unit No.'] == 4,'Total with VAT, USD'].sum()
    merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'total_cost_vat'] = (merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_vat_u0'] +  
                                                                                     merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_vat_u1'] + 
                                                                                     merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_vat_u2'] + 
                                                                                     merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_vat_u3'] + 
                                                                                     merged_df_final.loc[merged_df_final['Номерконтракта'] == val ,'cost_vat_u4'])
    del df
    
    #df2 = pd.concat([df2, filtered_df], ignore_index=True )

# df2[df2['Unit No.'] == 0]
# df2['Unit No.'].unique()
merged_df_final

,№,Лот,Номерконтракта,Дата контракта,Поставщик,Предмет поставки,Стоимость контракта,Предусмотренный аванс,Тонн,Смонтировано,...,cost_u2,cost_u3,cost_u4,total_cost,cost_vat_u0,cost_vat_u1,cost_vat_u2,cost_vat_u3,cost_vat_u4,total_cost_vat
0,412,AKU-21-021/021m,FT-04-21-889,2021-10-25,КОНЦЕРН ТИТАН-2 АО,Поставка трубопроводов турбинного острова для ...,1.209500e+08,3.628500e+07,NaN,NaN,...,3.023750e+07,30237500.0,30237500.0,1.209500e+08,0.000000e+00,3.023750e+07,3.023750e+07,30237500.0,30237500.0,1.209500e+08
1,413,AKU-20-207/207м,FT-04-21-626,2021-06-15,КОНЦЕРН ТИТАН-2 АО,Трубопроводы ядерного острова,5.591670e+07,1.677501e+07,NaN,NaN,...,2.793919e+07,0.0,0.0,5.591670e+07,0.000000e+00,2.797751e+07,2.793919e+07,0.0,0.0,5.591670e+07
2,414,AKU-21-113/113м,FT-04-22-1692,2022-12-12,КОНЦЕРН ТИТАН-2 АО,трубопроводы турбинного острова,3.985523e+07,9.060729e+06,NaN,NaN,...,1.510093e+07,0.0,0.0,3.020243e+07,0.000000e+00,1.992791e+07,1.992733e+07,0.0,0.0,3.985523e+07
3,415,AKU-22-282/282м,FT-04-23-665,2023-05-04,КОНЦЕРН ТИТАН-2 АО,трубопроводы турбинного острова,3.824765e+07,9.037067e+06,NaN,NaN,...,1.536751e+07,0.0,0.0,3.075823e+07,0.000000e+00,1.913798e+07,1.910967e+07,0.0,0.0,3.824765e+07
4,416,AKU-20-273/273м,FT-04-21-642,2021-06-30,КОНЦЕРН ТИТАН-2 АО,"Трубопроводов LAB, LBA",3.432932e+07,1.029880e+07,NaN,NaN,...,1.718242e+07,0.0,0.0,3.432932e+07,0.000000e+00,1.714690e+07,1.718242e+07,0.0,0.0,3.432932e+07
5,417,AKU-22-281/281м,FT-04-23-664,2023-05-02,КОНЦЕРН ТИТАН-2 АО,трубопроводы турбинного острова,3.027153e+07,8.071123e+06,NaN,NaN,...,1.336192e+07,0.0,0.0,2.690374e+07,0.000000e+00,1.524803e+07,1.502350e+07,0.0,0.0,3.027153e+07
6,418,AKU-19-058/2/2м,FT-04-21-890,2021-10-18,КОНЦЕРН ТИТАН-2 АО,Поставка трубопроводов ядерного острова для со...,1.352748e+07,4.058243e+06,NaN,NaN,...,1.352748e+07,0.0,0.0,1.352748e+07,0.000000e+00,0.000000e+00,1.352748e+07,0.0,0.0,1.352748e+07
7,419,AKU-22-046/046м,FT-04-22-1685,2022-12-13,КОНЦЕРН ТИТАН-2 АО,"трубопроводы зданий 01UYX, 00UYB, 00UFC",3.213636e+06,9.640909e+05,NaN,NaN,...,0.000000e+00,0.0,0.0,2.857249e+06,3.274509e+06,0.000000e+00,0.000000e+00,0.0,0.0,3.274509e+06
8,420,AKU-19-058,FT-04-20-76,2020-03-20,ТИТАН ТЕХНОЛОДЖИ ПАЙПЛАЙН ООО,Трубопроводы ЯО,1.247274e+07,3.741823e+06,NaN,NaN,...,0.000000e+00,0.0,0.0,1.396713e+07,0.000000e+00,1.396713e+07,0.000000e+00,0.0,0.0,1.396713e+07
9,454,AKU-22-283м,FT-04-23-912,2023-09-13,КОНЦЕРН ТИТАН-2 АО,трубопроводы турбинного острова,4.035195e+07,9.452929e+06,NaN,NaN,...,1.575488e+07,0.0,0.0,3.150976e+07,0.000000e+00,2.017603e+07,2.017593e+07,0.0,0.0,4.035195e+07


In [155]:
merged_df_final.to_excel(r'C:\Users\yuriy\Desktop\equipment\non project works\трубы_+cost.xlsx', index=False, float_format="%.4f")

In [ ]:
df = df_eqp[df_eqp['No Contract'] == 'FT-04-23-665']
filtered_df = df[['Unit No.', ' Net weight  total, kg', 'Total without VAT, USD', 'Total with VAT, USD']]

